In [1]:
import cvxopt
import numpy as np
import matplotlib.pyplot as plt

In [2]:
P = cvxopt.matrix(np.array([[1,0],[0,0]]),tc='d')
q = cvxopt.matrix(np.array([3,4]),tc='d')
g = cvxopt.matrix(np.array([[-1,0],[0,-1],[-1,-3],[2,5],[3,4]]),tc='d')
h = cvxopt.matrix(np.array([0,0,-15,100,80]),tc='d')

(2, 2)


In [4]:
sol = cvxopt.solvers.qp(P,q,g,h)
print(sol['x'])

     pcost       dcost       gap    pres   dres
 0:  1.0780e+02 -7.6366e+02  9e+02  0e+00  4e+01
 1:  9.3245e+01  9.7637e+00  8e+01  6e-17  3e+00
 2:  6.7311e+01  3.2553e+01  3e+01  6e-17  1e+00
 3:  2.6071e+01  1.5068e+01  1e+01  9e-17  7e-01
 4:  3.7092e+01  2.3152e+01  1e+01  8e-17  4e-01
 5:  2.5352e+01  1.8652e+01  7e+00  9e-17  3e-16
 6:  2.0062e+01  1.9974e+01  9e-02  1e-16  2e-16
 7:  2.0001e+01  2.0000e+01  9e-04  2e-16  0e+00
 8:  2.0000e+01  2.0000e+01  9e-06  9e-17  3e-16
Optimal solution found.
[ 7.13e-07]
[ 5.00e+00]



In [5]:
print(sol['x'][0])

7.131816408857276e-07


In [9]:
P = (2.0)*cvxopt.matrix(np.array([[2, 0.5],[0.5, 1]]), tc='d')
q = cvxopt.matrix(np.array([1,1]), tc='d')
G = cvxopt.matrix(np.array([[-1.0, 0.0],[0.0, -1.0]]), tc='d')
h = cvxopt.matrix(np.array([0,0]), tc='d')
A = cvxopt.matrix(np.array([1.0, 1.0]).reshape((1,2)), tc='d')
b = cvxopt.matrix(np.array([1.0]),tc='d')

In [10]:
sol = cvxopt.solvers.qp(P,q,G,h,A,b)
print(sol['x'])

     pcost       dcost       gap    pres   dres
 0:  1.8889e+00  7.7778e-01  1e+00  2e-16  2e+00
 1:  1.8769e+00  1.8320e+00  4e-02  1e-16  6e-02
 2:  1.8750e+00  1.8739e+00  1e-03  1e-16  5e-04
 3:  1.8750e+00  1.8750e+00  1e-05  0e+00  5e-06
 4:  1.8750e+00  1.8750e+00  1e-07  2e-16  5e-08
Optimal solution found.
[ 2.50e-01]
[ 7.50e-01]



In [11]:
print(sol['x'][0])
print(sol['x'][1])

0.2500000952702475
0.7499999047297525
